## روند آموزش یک Language Model (به‌سبک GPT، ساده‌شده)

| گام | توضیح | شکل (ابعاد نمونه‌ای) |
|-----|-------|-----------------------|
| 1. ساخت واژگان (Vocabulary) | تمام متن توکنایز می‌شود؛‌ واژگان با اندازهٔ V (مثلاً 50k) شکل می‌گیرد و برای هر توکن یک بردار در ماتریس یادگیری‌پذیرِ embedding ساخته می‌شود. | E ∈ ℝ^{V × d_model} |
| 2. تبدیل متن به ID | متن → دنبالهٔ بلندی از اعداد: 17, 328, 5, 42,… | بردار ۱بعدی خیلی بلند |
| 3. بُرش به بلوک‌های طول ثابت | با context_length = 32 دنباله را به قطعات 32تایی می‌بُریم: <br>`[t0…t31]`, [t32…t63], … | اگر 1024 توکن داشته باشیم → 32 بلوک <br>`input.shape = (32, 32)` |
| 4. ساخت ورودی/هدف (shift-one) | برای هر بلوک: <br>`inputs = [t0…t30]`   <br>`targets = [t1…t31]`  | inputs, targets ∈ ℤ^{batch, seq_len} |
| 5. Embedding + Position | x = E[inputs] + P  ⇒ شکل (batch, seq_len, d_model) |
| 6. عبور از لایه‌های ترنسفورمر | خروجی با همان شکل برمی‌گردد. |
| 7. نگاشت به فضای واژگان | logits = x @ Wᵀ + b  با ابعاد (batch, seq_len, V) |
| 8. محاسبۀ Loss | Cross-Entropy بین logits و targets. |
| 9. به‌روزرسانی وزن‌ها | Optimizer تمام پارامترهای یادگیری‌پذیر (`E`, ترنسفورمرها, W, `b`) را در مرحلهٔ backward آپدیت می‌کند. |

---

### نکته‌های کلیدی

* **Embedding Matrix فقط به V × d_model وابسته است**؛ اندازهٔ کتاب‌ها روی سایز آن اثر ندارد، بلکه روی تعداد batch‌ها اثر می‌گذارد.  
* Shift-one باعث می‌شود مدل در هر موقعیت، توکن بعدی را پیش‌بینی کند.  
* پارامترها یک‌بار در __init__ ساخته می‌شوند؛ در هر گام آموزش طی backward به‌روزرسانی می‌شوند و در فراخوانی‌های بعدی forward خودبه‌خود مقدار جدید دارند.  
* در صورت نیاز به دادهٔ بیشتر می‌توان stride را کوچک‌تر از context_length گرفت (پنجره‌های هم‌پوشان).

> «یک زبان-مدل، متن را به قطعات طول ثابت می‌بُرد، ورودی و خروجیِ شیفت‌شده می‌سازد، از شبکه عبور می‌دهد، روی logits softmax + cross-entropy می‌زند و وزن‌های embedding و ترنسفورمر را با backprop یاد می‌گیرد.»

<h1 style="color:rgb(253, 173, 119);">2.1 Tokenization</h1>
<span style="color:rgb(255, 255, 255);">-----------------------------------------</span>

<span style="color:rgb(253, 181, 229);">* Downlaod Dataset from github:</span>
1.</br>import urllib.request library
</br>
</br>2. Go to Github
</br>
</br>3. Select Raw
</br>
</br>4. Copy Address

In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
filename = "the-verdict.txt"
urllib.request.urlretrieve(url, filename)
print("Download Compeleted")

Download Compeleted


<span style="color:rgb(253, 181, 229);">* Open and Read the-verdict.txt</span>


In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
 raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])


Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


<h1 style="color:rgb(253, 173, 119);">2.2 Goal:</h1>
Our goal is to <span style="color:rgb(245, 142, 150);">tokenize</span> this 20,479-character short story into</br>
<span style="color:rgb(245, 142, 150);">individual words and special characters</span> that we can then</br>
turn into <span style="color:rgb(245, 142, 150);">embeddings</span>  for LLM training.
</br> --------------------------------------------------------------------------------------


<span style="color:rgb(253, 181, 229);">* Simple split e tokenization scheme model:</span>


In [3]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


<span style="color:rgba(249, 244, 248, 0.83);">somewords are still connected to punctuation characters that</br>
we want to have as separate list entries and make all lowercase
</br> and delete whitespace ---> r.split(r'([:;.,?!{}[]... -- _ __]|\\s)',text)
</br></br>it means:
1. r'()' -> for recognize ponctuation in a text and 
</br>2.|\\s for ignore whitespace
</span>


In [4]:
result2 = re.split(r'([,.]|\\s)', text)
print(result2)

['Hello', ',', ' world', '.', ' This', ',', ' is a test', '.', '']


In [5]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)


['Hello', ',', 'world', '.', 'Is this', '--', 'a test', '?']


In [6]:
preprocessed = re.findall(r"\b\w+(?:[-']\w+)*\b|[.,:;!?\"()\[\]—–-]", raw_text)

print("Total number of character:", len(preprocessed))
print(preprocessed[:30])

Total number of character: 4578
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '-', '-', 'though', 'a', 'good', 'fellow', 'enough', '-', '-', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that']


<h1 style="color:rgb(253, 173, 119);">2.3 Converting tokens into token IDs
</h1>
<span style="color:rgb(255, 255, 255);">-----------------------------------------</span>

<span style="color:rgb(245, 142, 150);">convert tokens from a python string to an integer to produce the token IDs:</span></br>
</br>2. build our vocabulary first</br>
</br>3. mapping each unique word with this vocabulary

<h1 style="color:rgb(252, 126, 42); font-size:20px">2.3.1 sort them alphabetically
to determine the vocabulary size: 
</h1>

In [7]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)
print(all_words)

1155
['!', '"', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft', 'Destroyed', 'Devonshire', "Don't", 'Dubarry_', 'Emperors', 'Florence', 'For', 'Gallery', 'Gideon', 'Gisburn', "Gisburn's", 'Gisburns', 'Grafton', 'Greek', 'Grindle', "Grindle's", 'Grindles', 'HAD', 'Had', 'Hang', 'Has', 'He', 'Her', 'Hermia', "Hermia's", 'His', 'How', 'I', "I'd", "I'll", "I've", 'If', 'In', 'It', "It's", 'Jack', "Jack's", 'Jove', 'Just', 'Lord', 'Made', 'Miss', "Money's", 'Monte', 'Moon-dancers', 'Mr', 'Mrs', 'My', 'Never', 'No', 'Now', 'Nutley', 'Of', 'Oh', 'On', 'Once', 'Only', 'Or', 'Perhaps', 'Poor', 'Professional', 'Renaissance', 'Rickham', 'Riviera', 'Rome', 'Russian', 'Sevres', 'She', "She's", 'Stroud', "Stroud's", 'Strouds', 'Suddenly', 'That', "That's", 'The', 'Then', 'There', 'They', 'This', 'Those', 'Though', 'Thwing', "Thwing's", 'Thwings', 'To', 'Usually', 'Ve

In [8]:
vocab = {}
i = 0
for word in all_words:
    vocab[word] = i
    i += 1

print(vocab)

{'!': 0, '"': 1, '(': 2, ')': 3, ',': 4, '-': 5, '.': 6, ':': 7, ';': 8, '?': 9, 'A': 10, 'Ah': 11, 'Among': 12, 'And': 13, 'Are': 14, 'Arrt': 15, 'As': 16, 'At': 17, 'Be': 18, 'Begin': 19, 'Burlington': 20, 'But': 21, 'By': 22, 'Carlo': 23, 'Chicago': 24, 'Claude': 25, 'Come': 26, 'Croft': 27, 'Destroyed': 28, 'Devonshire': 29, "Don't": 30, 'Dubarry_': 31, 'Emperors': 32, 'Florence': 33, 'For': 34, 'Gallery': 35, 'Gideon': 36, 'Gisburn': 37, "Gisburn's": 38, 'Gisburns': 39, 'Grafton': 40, 'Greek': 41, 'Grindle': 42, "Grindle's": 43, 'Grindles': 44, 'HAD': 45, 'Had': 46, 'Hang': 47, 'Has': 48, 'He': 49, 'Her': 50, 'Hermia': 51, "Hermia's": 52, 'His': 53, 'How': 54, 'I': 55, "I'd": 56, "I'll": 57, "I've": 58, 'If': 59, 'In': 60, 'It': 61, "It's": 62, 'Jack': 63, "Jack's": 64, 'Jove': 65, 'Just': 66, 'Lord': 67, 'Made': 68, 'Miss': 69, "Money's": 70, 'Monte': 71, 'Moon-dancers': 72, 'Mr': 73, 'Mrs': 74, 'My': 75, 'Never': 76, 'No': 77, 'Now': 78, 'Nutley': 79, 'Of': 80, 'Oh': 81, 'On': 8

In [9]:
"""
enumerate-> 
هم میخوای بدونی عنصر چیه مثلا چه کلمه ایه و هم موقعیتش کجاس
"""
for key, item in enumerate(vocab.items()):
    # print(item)
    if key >= 52:
        break

<h1 style="color:rgb(253, 173, 119);">Listing 2.3 Implementing a simple text tokenizer
</h1>
#1 Stores the vocabulary as a class attribute for access in the encode
and decode methods</br>
#2 Creates an inverse vocabulary that maps token IDs back to the
original text tokens</br>
#3 Processes input text into token IDs</br>
#4 Converts token IDs back into text</br>
#5 Removes spaces before the specified punctuation</br>
<span style="color:rgb(255, 255, 255);">-----------------------------------------</span></br>
<h1 style="color:rgb(253, 173, 119);">2.4 Adding special context tokens : </h1>
</br>add an <|unk|> token to represent new and</br>
unknown words that were not part of the training data and thus not part</br>
of the existing vocabulary. Furthermore, we add an <|endoftext|> token</br>
that we can use to separate two unrelated text sources.


In [10]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        preprocessed_org = re.findall(r"\b\w+(?:[-']\w+)*\b|[.,:;!?\"()\[\]—–-]", vocab)
        sorted_word = sorted(list(set(preprocessed_org)))
        sorted_word.extend(["<|endoftext|>", "<|unk|>"])
        vocab = {token:integer for integer, token in enumerate(sorted_word)}
        print(vocab)
        for i, item in enumerate(list(vocab.items())[-5:]):
            print(item)

        self.str_to_int = vocab 
        self.int_to_str = {i:s for s,i in vocab.items()}
# For new text
    def encode(self, text):
        preprocessed = re.findall(r"<\|.*?\|>|[\w']+|[.,!?;]", text)
        ids = []
        for token in preprocessed:
            if token in self.str_to_int:
                ids.append(self.str_to_int[token])
            else:
                ids.append(self.str_to_int[ "<|unk|>" ])
        
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        
        text = re.sub(r'\s+([\\])', r'\\1', text)
        return text


<span style="color:rgb(253, 173, 119); font-size:20px">Wharton’s short story to try it out in practice:</span></br>
The preceding code prints the following token IDs:



In [11]:
tokenizer = SimpleTokenizerV1(vocab=raw_text)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
total_text = "<|endoftext|>".join((text1, text2))

ids = tokenizer.encode(text=total_text)
print(ids)

{'!': 0, '"': 1, '(': 2, ')': 3, ',': 4, '-': 5, '.': 6, ':': 7, ';': 8, '?': 9, 'A': 10, 'Ah': 11, 'Among': 12, 'And': 13, 'Are': 14, 'Arrt': 15, 'As': 16, 'At': 17, 'Be': 18, 'Begin': 19, 'Burlington': 20, 'But': 21, 'By': 22, 'Carlo': 23, 'Chicago': 24, 'Claude': 25, 'Come': 26, 'Croft': 27, 'Destroyed': 28, 'Devonshire': 29, "Don't": 30, 'Dubarry_': 31, 'Emperors': 32, 'Florence': 33, 'For': 34, 'Gallery': 35, 'Gideon': 36, 'Gisburn': 37, "Gisburn's": 38, 'Gisburns': 39, 'Grafton': 40, 'Greek': 41, 'Grindle': 42, "Grindle's": 43, 'Grindles': 44, 'HAD': 45, 'Had': 46, 'Hang': 47, 'Has': 48, 'He': 49, 'Her': 50, 'Hermia': 51, "Hermia's": 52, 'His': 53, 'How': 54, 'I': 55, "I'd": 56, "I'll": 57, "I've": 58, 'If': 59, 'In': 60, 'It': 61, "It's": 62, 'Jack': 63, "Jack's": 64, 'Jove': 65, 'Just': 66, 'Lord': 67, 'Made': 68, 'Miss': 69, "Money's": 70, 'Monte': 71, 'Moon-dancers': 72, 'Mr': 73, 'Mrs': 74, 'My': 75, 'Never': 76, 'No': 77, 'Now': 78, 'Nutley': 79, 'Of': 80, 'Oh': 81, 'On': 8

<span style="color:rgb(249, 236, 228); font-size:20px">Next, let’s see whether we can turn these token IDs back</br>
into text using the decode method:</span>

In [12]:
print(tokenizer.decode(ids=ids))

<|unk|> , do you like tea ? <|endoftext|> In the sunlit terraces of the <|unk|> .


<h1 style="color:rgb(253, 173, 119);">2.5 Byte pair encodin : 
</h1>

In [13]:
from importlib.metadata import version
import tiktoken

In [14]:
tokenizer = tiktoken.get_encoding("gpt2")

<span style="color:rgb(246, 226, 212); font-size:20px">The usage of this tokenizer is similar to the SimpleTokenizerV2
</br>
we implemented previously via an encode method :</span>

In [15]:
text = (
 "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
 "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)


[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


<span style="color:rgb(246, 226, 212); font-size:20px">convert the token IDs back into text using the
</br>
decode method, similar to our SimpleTokenizerV2:
 :</span>


In [16]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [17]:
text = "Akwirw ier"

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

for ids in integers:
    print(tokenizer.decode_single_token_bytes(ids))

print(tokenizer.decode(integers))

[33901, 86, 343, 86, 220, 959]
b'Ak'
b'w'
b'ir'
b'w'
b' '
b'ier'
Akwirw ier


<h1 style="color:rgb(253, 173, 119);">2.6 Data sampling with a sliding
window : 
</h1></br>
<span style="color:rgb(246, 226, 212); font-size:20px">generate the input–target pairs required for training an LLM</span>

--------------------------------------------------------------------------
## 🧠 Sliding-Window و Batching در آموزش GPT-style Language Models

فرض کنید جمله‌ی زیر رو داریم:

بعد از تبدیل به توکن (مثلاً با BPE)، به شکل زیر درمیاد:

---

### 🔁 Sliding Window — ساخت input و target

فرض کنیم طول پنجره (window) برابر ۳ باشه (`max_length = 3`).  
در این روش، از دنباله‌ی توکن‌ها، ورودی و خروجی‌هایی می‌سازیم که مدل بتونه توکن بعدی رو پیش‌بینی کنه:

| Input               | Target              |
|--------------------|---------------------|
| [I, love, you]     | [love, you, so]     |
| [love, you, so]    | [you, so, much]     |
| [you, so, much]    | [so, much, baby]    |
| [so, much, baby]   | [much, baby, kiss]  |
| [much, baby, kiss] | [baby, kiss, me]    |
| [baby, kiss, me]   | [kiss, me, please]  |

در واقع، خروجی فقط یک توکن شیفت‌یافته از ورودی هست. هدف مدل اینه که برای هر موقعیت، توکن بعدی رو پیش‌بینی کنه.

---

### 🧱 Batching — آموزش موازی

برای افزایش کارایی، این جفت‌ها رو با batch_size = 3 در قالب تنسور به مدل می‌دیم:

#### 🔹 Batch 1:
```python
inputs  = [[I, love, you],
           [love, you, so],
           [you, so, much]]

targets = [[love, you, so],
           [you, so, much],
           [so, much, baby]]

inputs  = [[so, much, baby],
           [much, baby, kiss],
           [baby, kiss, me]]

targets = [[much, baby, kiss],
           [baby, kiss, me],
           [kiss, me, please]]

In [18]:
encoding_text = tokenizer.encode(raw_text)
print(len(encoding_text))

5145


In [19]:
enc_sample = encoding_text[50:]
print(enc_sample)

[290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686, 41976, 13, 357, 10915, 314, 2138, 1807, 340, 561, 423, 587, 10598, 393, 28537, 2014, 198, 198, 1, 464, 6001, 286, 465, 13476, 1, 438, 5562, 373, 644, 262, 1466, 1444, 340, 13, 314, 460, 3285, 9074, 13, 46606, 536, 5469, 438, 14363, 938, 4842, 1650, 353, 438, 2934, 489, 3255, 465, 48422, 540, 450, 67, 3299, 13, 366, 5189, 1781, 340, 338, 1016, 284, 3758, 262, 1988, 286, 616, 4286, 705, 1014, 510, 26, 475, 314, 836, 470, 892, 286, 326, 11, 1770, 13, 8759, 2763, 438, 1169, 2994, 284, 943, 17034, 318, 477, 314, 892, 286, 526, 383, 1573, 11, 319, 9074, 13, 536, 5469, 338, 11914, 11, 33096, 663, 4808, 3808, 62, 355, 996, 484, 547, 12548, 287, 281, 13079, 410, 12523, 286, 22353, 13, 843, 340, 373, 407, 691, 262, 9074, 13, 536, 48819, 508, 25722, 276, 13, 11161, 407, 262, 40123, 18113, 544, 9325, 701, 11, 379, 262, 938, 402, 1617, 261, 12917, 905, 11, 5025, 502, 878, 402, 271, 10899, 338, 366, 31640, 12, 67, 20811, 1, 284, 910, 11, 351, 10

In [20]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"X:  {x}")
print(f"y:      {y}")

X:  [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [21]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "--------->", desired)
    print(tokenizer.decode(context), "--------->", tokenizer.decode([desired]))

[290] ---------> 4920
 and --------->  established
[290, 4920] ---------> 2241
 and established --------->  himself
[290, 4920, 2241] ---------> 287
 and established himself --------->  in
[290, 4920, 2241, 287] ---------> 257
 and established himself in --------->  a


<h1 style="color:rgb(253, 173, 119);"> Dataset for Data sampling with a sliding : 
</h1></br>
The GPTDatasetV1 class is based on the PyTorch Dataset class
and defines how individual</br> rows are fetched from the
dataset, where each row consists of a number of token IDs</br>
(based on a max_length) assigned to an input_chunk tensor. The
target_ chunk tensor</br> contains the corresponding targets.


------------------------------------------------------------------------------

## 📄 کلاس GPTDatasetV1 — آماده‌سازی داده‌ی آموزشی برای GPT

In [22]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids  = []        # همه‌ی توالی‌های ورودی (X)
        self.target_ids = []        # همه‌ی توالی‌های هدف   (Y)

        # 1️⃣  کل متن را توکنیزه می‌کنیم
        token_ids = tokenizer.encode(txt)

        # 2️⃣  با Sliding-Window متن را به سکانس‌های هم‌پوشان تقسیم می‌کنیم
        #     طول هر سکانس = max_length
        #     هر بار به‌اندازه‌ی  stride  جلو می‌رویم
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk  = token_ids[i            : i + max_length]
            target_chunk = token_ids[i + stride        : i + max_length + stride]  # یک توکن شیفت جلوتر

            # 3️⃣  تبدیل هر قطعه به Tensor برای PyTorch
            self.input_ids .append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    # 4️⃣  تعداد کل نمونه‌ها (برای DataLoader)
    def __len__(self):
        return len(self.input_ids)

    # 5️⃣  برگرداندن یک جفت (X, Y) بر اساس اندیس
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [23]:
def create_dataloader_v1(
        txt: str,
        batch_size: int = 4,
        max_length: int = 256,
        stride: int = 128,
        shuffle: bool = True,
        drop_last: bool = True,
        num_workers: int = 0):

    # -------------------------------------------
    # 1) ساخت توکنایزر BPE (tikToken = سریع و سبک)
    # -------------------------------------------
    tokenizer = tiktoken.get_encoding("gpt2")  # می‌توان مدل دلخواه BPE را انتخاب کرد

    # ----------------------------------------------------------
    # 2) تبدیل کل متن به دیتاست Sliding-Window (GPTDatasetV1)
    # ----------------------------------------------------------
    dataset = GPTDatasetV1(
        txt=txt,
        tokenizer=tokenizer,
        max_length=max_length,
        stride=stride)

    # ----------------------------------------------------------
    # 3) ساخت DataLoader رسمی PyTorch
    #    - drop_last : جلوگیری از بتچ کوچک
    #    - shuffle   : برای آموزش بهتر
    #    - num_workers: سرعت بیشتر هنگام CPU-preprocessing
    # ----------------------------------------------------------
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    # برگشت آبجکت آمادهٔ مصرف توسط حلقهٔ آموزش
    return dataloader

In [24]:
#iter example for test
itera = [1, 2, 3]
my_iter = iter(itera)
data = next(my_iter)
print(data)
data = next(my_iter)
print(data)

1
2


In [25]:
dataloader = create_dataloader_v1(
 raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

#1 Converts dataloader into a Python iterator to fetch the next entry via|>
#|>Python’s built-in next() function
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\\n", targets)


Inputs:
 tensor([[  40,  367, 2885, 1464]])

Targets:\n tensor([[ 367, 2885, 1464, 1807]])


## 2.7 Creating Token Embeddings

در این بخش، هدف ما ایجاد embedding برای توکن‌های عددی است که نماینده‌ی کلمات یا نشانه‌ها در یک مدل زبان هستند.

### 📌 مرحله 1: تعریف ورودی

```python
input_ids = torch.tensor([2, 3, 5, 1])

In [26]:
input_ids = torch.tensor([2, 3, 5, 1])

vocab_size = 6
output_dim = 3

In [27]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [28]:
print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## 2.8 Encoding Word Positions 🧭

### 🧠 مشکل اصلی:
در مدل‌های زبان بزرگ (LLMs)، لایه‌ی embedding توکن‌ها را به بردارهایی نگاشت می‌کند. اما یک نکته‌ی مهم اینجاست:

> ❗ Self-attention به‌تنهایی ترتیب یا مکان توکن‌ها را نمی‌فهمد!

مثلاً توکن ۵ همیشه به یک بردار مشخص نگاشت می‌شود، بدون توجه به این‌که در اول جمله آمده یا آخر.

---

### 🔄 رفتار لایه‌ی embedding

لایه‌ی embedding به‌صورت زیر عمل می‌کند:

```text
Token IDs = [2, 3, 5, 2]  # معادل جمله‌ی فرضی "fox jumps over fox"

Embedding Weight Matrix:
ID 2 → [1.2753, -0.2010, -0.1606]
ID 3 → [0.4015,  0.9666, -1.1481]
ID 5 → [2.8400, -0.7849, -1.4096]

[1.2753, -0.2010, -0.1606]  ← fox (اول)
[0.4015,  0.9666, -1.1481]  ← jumps
[2.8400, -0.7849, -1.4096]  ← over
[1.2753, -0.2010, -0.1606]  ← fox (دوباره!)

In [29]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [30]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, 
    max_length=max_length, 
    stride=max_length, 
    shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\\n", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:\n torch.Size([8, 4])


In [31]:
token_embedding = token_embedding_layer(inputs)
print(token_embedding.shape)

torch.Size([8, 4, 256])


### 📍 Position Embedding در PyTorch

در معماری Transformer**، چون هیچ ترتیب ذاتی‌ای بین توکن‌ها وجود ندارد، لازم است **موقعیت (Position) هر توکن به مدل داده شود تا بتواند ترتیب کلمات را درک کند. این کار از طریق Position Embedding انجام می‌شود.

---

#### 🛠️ ساخت Position Embedding

در این مثال ساده، مراحل زیر انجام می‌شود:

- 🔢 context_length برابر است با حداکثر تعداد توکن‌هایی که مدل در یک ورودی می‌بیند.
- 🧱 یک لایه‌ی Embedding به‌صورت context_length × output_dim ساخته می‌شود.
  - این لایه به هر موقعیت از 0 تا context_length - 1 یک بردار ویژگی (embedding) اختصاص می‌دهد.
- 📊 با استفاده از torch.arange(context_length) یک آرایه از موقعیت‌ها ساخته می‌شود (مثلاً `[0, 1, 2, 3]`).
- 🧬 این آرایه به لایه‌ی embedding داده می‌شود تا بردار embedding مربوط به هر موقعیت تولید شود.

---

#### ✅ خروجی نمونه:

```python
torch.Size([4, 256])

In [32]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [33]:
input_embeddings = token_embedding + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [34]:
text = "Your journey starts with one step "

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

integers_torch = torch.tensor(integers).unsqueeze(0)
print(integers_torch)

[7120, 7002, 4940, 351, 530, 2239, 220]
tensor([[7120, 7002, 4940,  351,  530, 2239,  220]])


In [ ]:
print(token_embedding_layer(integers_torch))

tensor([[[ 1.2533, -0.0438, -0.8669,  ...,  0.6702,  0.2240, -0.5718],
         [ 0.0643, -0.3555,  0.0845,  ..., -0.1261, -0.4285, -0.1846],
         [ 0.8342,  0.5134,  0.6161,  ..., -1.2478, -0.3091, -0.4559],
         ...,
         [-0.6179,  0.2314, -0.5075,  ..., -0.4943,  0.2457,  0.3168],
         [-0.8742,  0.9597, -0.4078,  ...,  1.4336, -0.6442,  1.0769],
         [-1.4157, -0.2557,  1.6429,  ..., -1.0009, -0.8753, -0.5352]]],
       grad_fn=<EmbeddingBackward0>)


: 

---

💡 چرا با وجود داشتن شناسه‌های واژه، باز هم به برش پنجره‌ای نیاز داریم؟

---

### 🔹 تفاوت بین سه مرحلهٔ اصلی در مدل زبانی

| مرحله | هدف | خروجی |
|-------|------|--------|
| رمزنگاری واژه‌ها | تبدیل واژه‌ها به اعداد | برداری از اعداد مانند [۱۲، ۳۱، ۴۶۷، ...] |
| تعبیهٔ برداری | نگاشت هر عدد به یک بردار چگال | آرایه‌ای سه‌بعدی از بردارهای معنا‌دار |
| آماده‌سازی داده‌ها | ساخت داده‌های آموزشی | مجموعه‌ای از ورودی و خروجی برای مدل |

🔸 مرحلهٔ سوم، همان جایی‌ست که «برش پنجره‌ای» انجام می‌شود.

---

### 🔹 دلایل نیاز به برش پنجره‌ای

۱. محدودیت حافظه  
مدل نمی‌تواند یک متن بسیار طولانی را به‌صورت یک‌جا پردازش کند. پس باید متن را به بخش‌های کوچکتر برش دهیم.

۲. افزایش دادهٔ آموزشی  
با برش‌های همپوشان (مثلاً با جابجایی یک واژه در هر برش)، می‌توان از یک متن، هزاران نمونهٔ آموزشی ساخت.

۳. ساخت هدف پیش‌بینی توکن بعدی  
برای آموزش مدل‌های زبانی، ورودی یک بخش از متن است و خروجی، همان متن با یک واژه جابه‌جا شده است:
ورودی  ← [تو، مرد، خوبی، هستی]  
خروجی ← [مرد، خوبی، هستی، ؟]

۴. تنوع در یادگیری و جلوگیری از بیش‌برازش (Overfitting)  
مدل باید از نمونه‌های گوناگون یاد بگیرد. این تنوع با برش‌های متعدد و تصادفی حاصل می‌شود.

---

### 🔹 اگر متن کامل را یک‌جا وارد کنیم چه می‌شود؟

- ممکن است مدل کار کند، اما:
  - یادگیری بسیار کند و پرخطا خواهد بود.
  - امکان آموزش گروهی (Batch) را از دست می‌دهیم.
  - استفاده از حافظه به‌شدت زیاد می‌شود.
  - مدل احتمالاً به‌خوبی یاد نمی‌گیرد که «توکن بعدی» را پیش‌بینی کند.

---

### 🔹 نتیجه‌گیری

حتی وقتی تمام شناسه‌های واژه‌ها را در اختیار داریم، برای آموزش مؤثر، همچنان به تکنیک «برش پنجره‌ای» نیاز داریم. این کار کمک می‌کند تا مدل:

- بهتر یاد بگیرد،  
- با حافظهٔ محدود سازگار باشد،  
- و با حجم دادهٔ بیشتر آموزش ببیند.

---